In [60]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import sys
import string
import requests
import time

In [61]:
def player_info():
    players = []
    base_url = "http://www.basketball-reference.com/players/"
    
    for letter in string.ascii_lowercase:
        page = requests.get(base_url+letter)
        soup = BeautifulSoup(page.text,features = "lxml")
        table = soup.find('tbody')
        
        if table:
            for row in table.findAll('tr'):
                player_url = row.find('a')
                player_name = player_url.text
                player_page = player_url['href']
                cells = row.findAll('td')
                player_start_year = cells[0].text
                player_end_year = cells[1].text
                player_pos = cells[2].text
                player_height = cells[3].text
                player_weight =  cells[4].text
                player_bday = cells[5].text
                player_college = cells[6].text
                
                player_entry = {
                                    'url': player_page,
                                    'name': player_name,
                                    "from": player_start_year,
                                    "to": player_end_year,
                                    "pos": player_pos,
                                    "height": player_height,
                                    "weight": player_weight,
                                    "birthday": player_bday,
                                    "college": player_college
                                }
              
                players.append(player_entry)
    return pd.DataFrame(players)

In [157]:
get_basic_player_info = player_info()

TypeError: 'tuple' object is not callable

In [182]:
def player_detailed_info(url):
    personal = SoupStrainer('p')
    page_request = requests.get("https://www.basketball-reference.com"+url)
    soup = BeautifulSoup(page_request.text, features = "lxml", parse_only = personal)
    p = soup.findAll('p') 
    shooting_hand = None
    birth_place = None
    high_school = None
    pos_start = None
    draft = None
    for prow in p:
        if 'Shoots:' in prow.text:
            s = prow.text.replace('\n','').split(u"\u25AA")
            if len(s)>1:
                shooting_hand = s[1].split(':')[1]
        if 'Position:' in prow.text:
            s = prow.text.replace('\n','').split(u"\u25AA")
            if(len(s)>1):
                pos_start = s[0].split(':')[1]
        if 'Born:' in prow.text:
            s = prow.text.split(u'\xa0')
            if len(s[1])>1:
                birth_place = s[1].replace(",","")
        if 'High School:' in prow.text:
            s = prow.text.replace("\n",'').split(':')
            if len(s)>1:
                high_school = s[1].lstrip()
        if 'Draft:' in prow.text:
            print(prow.text)
                
                

In [183]:
player_detailed_info(get_basic_player_info['url'][3432])
get_basic_player_info['url'][1000]
len(get_basic_player_info)



  Draft:
  
Milwaukee Bucks, 1st round (22nd pick, 22nd overall), 1985 NBA Draft



4639